In [9]:
from nwpackageversions import PyPiReleaseManager, Session

release_manager : PyPiReleaseManager = PyPiReleaseManager()
session : Session = release_manager.fetch(package_name = "numpy", only_final_releases = True)
# release_manager.log_session(session = session)
# release_manager.log_releases(releases = session.releases)
# release_manager.log_list(lst = session.releases)
release_manager.log_list(lst = session.xml_items)

{ 'title': '2.1.2', 'link': 'https://pypi.org/project/numpy/2.1.2/', 'description': 'Fundamental package for array computing in Python', 'author': 'None', 'pubdate_str': 'Sat, 05 Oct 2024 18:28:18 GMT' }
{ 'title': '2.1.1', 'link': 'https://pypi.org/project/numpy/2.1.1/', 'description': 'Fundamental package for array computing in Python', 'author': 'None', 'pubdate_str': 'Tue, 03 Sep 2024 15:01:06 GMT' }
{ 'title': '2.0.2', 'link': 'https://pypi.org/project/numpy/2.0.2/', 'description': 'Fundamental package for array computing in Python', 'author': 'None', 'pubdate_str': 'Mon, 26 Aug 2024 20:04:14 GMT' }
{ 'title': '2.1.0', 'link': 'https://pypi.org/project/numpy/2.1.0/', 'description': 'Fundamental package for array computing in Python', 'author': 'None', 'pubdate_str': 'Sun, 18 Aug 2024 21:39:07 GMT' }
{ 'title': '2.1.0rc1', 'link': 'https://pypi.org/project/numpy/2.1.0rc1/', 'description': 'Fundamental package for array computing in Python', 'author': 'None', 'pubdate_str': 'Sun, 11